In [10]:
!pip install pandas
!pip install ipython-sql prettytable 

import prettytable

prettytable.DEFAULT = 'DEFAULT'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 173.4 MB/s eta 0:00:00


In [11]:
# Load the SQL magic extension
%load_ext sql

# Connect the %sql magic to the SQLite database (this is needed for %sql cells)
%sql sqlite:///FinalDB.db

# Import libraries
import pandas as pd
import sqlite3

# Regular Python SQLite connection (optional for pandas)
con = sqlite3.connect("FinalDB.db")


In [12]:
# Load CSVs into DataFrames
df1 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv')
df2 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv')
df3 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv')

# Save DataFrames to SQL tables
df1.to_sql('CENSUS_DATA', con, if_exists='replace', index=False)
df2.to_sql('CHICAGO_PUBLIC_SCHOOLS', con, if_exists='replace', index=False)
df3.to_sql('CHICAGO_CRIME_DATA', con, if_exists='replace', index=False)

533

#P1 Find the total number of crimes recorded in the CRIME table.

In [51]:
%sql SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


#P2List community area names and numbers with per capita income less than 11000

In [54]:
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


#P3 List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [50]:
%sql SELECT DISTINCT CASE_NUMBER FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%';


 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


#Code Explanation
#2. SELECT DISTINCT CASE_NUMBER: DISTINCT removes duplicates — so if the same CASE_NUMBER appears multiple times in the results, you only see it once. CASE_NUMBER is a column in your table — probably the unique ID for each reported crime.

P4: List all kidnapping crimes involving a child?

In [16]:
%sql SELECT DISTINCT CASE_NUMBER, PRIMARY_TYPE FROM CHICAGO_CRIME_DATA WHERE (DESCRIPTION LIKE '%CHILD%') AND (PRIMARY_TYPE LIKE '%KIDNAPPING%');

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE
HN144152,KIDNAPPING


In [55]:
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE (DESCRIPTION LIKE '%CHILD%') AND (PRIMARY_TYPE = 'KIDNAPPING');

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


P5: List the kind of crimes that were recorded at schools. (No repetitions)

In [35]:
%sql SELECT DISTINCT PRIMARY_TYPE FROM CHICAGO_CRIME_DATA WHERE (LOCATION_DESCRIPTION LIKE '%SCHOOL%');

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


P6: List the type of schools along with the average safety score for each type.


In [37]:
%%sql SELECT "Elementary, Middle, or High School",
    AVG(SAFETY_SCORE) AS average_safety_score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",average_safety_score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


P7: List 5 community areas with highest % of households below poverty line

In [39]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


P8: Which community area is most crime prone? Display the coumminty area number only.

In [46]:
%%sql
SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


P9: Use a sub-query to find the name of the community area with highest hardship index

In [41]:
#Option 1:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX = ( SELECT MAX(HARDSHIP_INDEX) from CENSUS_DATA);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [60]:
#Option 2:

%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX IN (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


P10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [49]:
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
Austin,25.0
